# Lista 06

## Imports

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
class newtonLinear():
    
    def __init__(self, bus_df, branch_df, referencia, perdas = False):
        
        self.bus_df    = bus_df
        self.branch_df = branch_df
        
        self.bus_len = len(self.bus_df)
        self.bra_len = len(self.branch_df)
        
        self.referencia = referencia
        self.perdas     = perdas
        
        
    def createMatrix(self):
        
        self.P     = np.zeros(self.bus_len, dtype=float)
        self.B     = np.zeros((self.bus_len, self.bus_len), dtype=float)
        self.B_inf = np.zeros((self.bus_len, self.bus_len), dtype=float)
        self.a     = np.zeros(self.bus_len, dtype=float)
        
        self.xm = np.zeros((self.bus_len, self.bus_len), dtype=float)
        
        if self.perdas:
            self.X  = np.zeros((self.bus_len, self.bus_len), dtype=float)
            self.R  = np.zeros((self.bus_len, self.bus_len), dtype=float)
        
        for i in range(self.bus_len):
            
            self.P[i]  = self.bus_df.loc[i+1,['Pg']].to_numpy(dtype=float)[0] - self.bus_df.loc[i+1,['Pc']].to_numpy(dtype=float)[0]
        
        for i in range(self.bra_len):
            
            k = self.branch_df.loc[i,['From']].to_numpy(dtype=int)[0]-1
            m = self.branch_df.loc[i,['To']].to_numpy(dtype=int)[0]-1

            r = self.branch_df.loc[i,['r']].to_numpy(dtype=float)[0]
            x = self.branch_df.loc[i,['x']].to_numpy(dtype=float)[0]

            self.xm[k, m] = 1/x
            self.xm[m, k] = 1/x
            
            if self.perdas:
            
                self.X[k, m] = x
                self.X[m, k] = x

                self.R[k, m] = r
                self.R[m, k] = r
            
        for i in range(self.bus_len):
            for j in range(self.bus_len):
                
                if i != j:
                    
                    self.B[i, j]     = - self.xm[i, j]
                    self.B_inf[i, j] = - self.xm[i, j]
                    
        for i in range(self.bus_len):
            
            soma = 0
            for j in range(self.bus_len):
                
                if i != j:
                    
                    soma += self.xm[i, j]
                    
            self.B[i, i]     = soma
            self.B_inf[i, i] = soma
            
        self.B_inf[self.referencia-1, self.referencia-1] = np.Inf
        
        if self.perdas:
            
            self.g = np.zeros((self.bus_len, self.bus_len), dtype=float)
            
            for i in range(self.bus_len):
                for j in range(self.bus_len):
                    
                    if i != j:
                        
                        if self.R[i, j]**2 + self.X[i, j]**2 != 0:
                        
                            self.g[i, j] = self.R[i, j]/(self.R[i, j]**2 + self.X[i, j]**2)
            
            
        
    def angleCal(self):
        
        self.b_inv = np.linalg.inv(self.B_inf)

        self.a = np.dot(self.b_inv, self.P)
        
        self.bus_df['Theta'] = self.a
        
        if self.perdas:    
            
            self.ad = np.zeros((self.bus_len, self.bus_len), dtype=float)
            
            for i in range(self.bus_len):
                for j in range(self.bus_len):
                    
                    if i != j:
            
                     self.ad[i, j] = self.a[i] - self.a[j]
        
    def powerCal(self):
        
        self.P = np.dot(self.B, self.a)
        Pg = np.zeros(self.bus_len, dtype=float)
        Pc = np.zeros(self.bus_len, dtype=float)
        
        
        if self.perdas:
            
            self.p1  = np.zeros(self.bus_len, dtype=float)
            self.pot = np.zeros(self.bus_len, dtype=float)
            self.Pp  = np.zeros((self.bus_len, self.bus_len), dtype=float)
            
            self.branch_df['Perdas'] = np.zeros(self.bra_len, dtype=float)
            
            for i in range(self.bus_len):
                
                soma = 0 
                for j in range(self.bus_len):
                    
                    if i != j:
                        
                        soma += -(self.g[i, j]*self.ad[i, j]**2)/2
                        
                self.p1[i] = soma
                
            self.pot = self.P + self.p1
            
            self.a = np.dot(self.b_inv, self.pot)
            self.P = np.dot(self.B, self.a)
            
            self.bus_df['Theta'] = self.a
            
            for i in range(self.bus_len):
                self.P[i] = self.P[i] - self.p1[i]
                
            for i in range(self.bus_len):
                for j in range(self.bus_len):
                    
                    if i != j:
                        self.Pp[i, j] = self.P[i] + self.P[j]
                        
                        
            for i in range(self.bra_len):
                
                k = self.branch_df.loc[i,['From']].to_numpy(dtype=int)[0]-1
                m = self.branch_df.loc[i,['To']].to_numpy(dtype=int)[0]-1
                
                self.branch_df.loc[i,['Perdas']] = self.Pp[k, m]
                
                
                
                        
        for i in range(self.bus_len):
            
            if self.P[i] > 0:
                
                Pg[i] = self.P[i]
                
            else:
                
                Pc[i] = np.abs(self.P[i])
                
        self.bus_df['Pg'] = Pg
        self.bus_df['Pc'] = Pc

## 1)

In [ ]:
rkm = 0.1
xkm = 1.0
bsh = 0.1
zkm = np.complex(rkm, xkm)
ykm = 1/zkm

vk = 138000/138000
vm = 138000/138000

Sb = 100000000

gkm = rkm/(rkm**2 + xkm**2)

graus   = [7.5, 15.0, 25.0]
tensoes = [1.0, 0.99, 0.98]
# theta_km = (graus*np.pi)/180

### a)

In [ ]:
for i, ang in enumerate(graus):
    vm = tensoes[i]
    theta_km = (ang*np.pi)/180
    
    PkmC = theta_km/xkm
    PmkC = -theta_km/xkm
    # print(Pkm, Pmk)
    
    PkmA = (vk**2)*ykm.real - vk*vm*(ykm.real*np.cos(theta_km) + ykm.imag*np.sin(theta_km))
    PmkA = (vm**2)*ykm.real - vk*vm*(ykm.real*np.cos(theta_km) - ykm.imag*np.sin(theta_km))
    
    print(f'Pkm:     {PkmA*100:0.6f} |  {PkmC*100:0.6f} | {100*(np.abs(PkmA-PkmC)/PkmC):0.2f}%')
    print(f'Pmk:    {PmkA*100:0.6f} | {PmkC*100:0.6f} | {100*(np.abs((PmkA-PmkC)/PmkC)):0.2f}%')
    print(f'Perdas:   {(PmkA+PkmA)*100:0.6f} |   {(gkm*(theta_km**2))*100:0.6f} | {100*(np.abs((PmkA+PkmA)-(gkm*(theta_km**2)))/(gkm*(theta_km**2))):0.2f}%')
    
    print("=======================================")
    

Pkm:     13.008090 |  13.089969 | 0.63%
Pmk:    -12.838681 | -13.089969 | 1.92%
Perdas:   0.169409 |   0.169651 | 0.14%
Pkm:     25.802396 |  26.179939 | 1.44%
Pmk:    -25.133417 | -26.179939 | 4.00%
Perdas:   0.668979 |   0.678603 | 1.42%
Pkm:     42.113637 |  43.633231 | 3.48%
Pmk:    -40.291491 | -43.633231 | 7.66%
Perdas:   1.822146 |   1.885009 | 3.33%


### b)

Pode-se perceber que o s valores obtidos para o método não linearizado estão bem próximos do método convencional, contudo, nota-se que o aumento do ângulo theta faz com que a diferença entre o c.c. e o c.a. aumentem, desse modo, conclui-se que o modelo c.c. funciona melhor como aproximação para valores pequenos de theta

## 2)

In [ ]:
b = [[ 1.5, -1, -0.5],
     [  -1,  2,   -1],
     [-0.5, -1,  1.5]]

b_a = [[    np.Inf, -1, -0.5],
       [        -1,  2,   -1],
       [      -0.5, -1,  1.5]]

b_b = [[ 1.5,          -1, -0.5],
       [  -1,      np.Inf,   -1],
       [-0.5,          -1,  1.5]]

pot_at = [0, 0.05, 0.015]


b_a    = np.asarray(b_a)
b_b    = np.asarray(b_b)
pot_at = np.asarray(pot_at)

angs = np.zeros(3)

### a)

In [ ]:
b_inv = np.linalg.inv(b_a)
print(b_inv)

angs  = np.dot(b_inv, pot_at)
print("Angulos:", angs)

[[0.   0.   0.  ]
 [0.   0.75 0.5 ]
 [0.   0.5  1.  ]]
Angulos: [0.    0.045 0.04 ]


In [ ]:
pot_at = np.dot(b, angs)
print("Potencia:", pot_at)

Potencia: [-0.065  0.05   0.015]


### b)

In [ ]:
b_inv = np.linalg.inv(b_b)
print(b_inv)

angs  = np.dot(b_inv, pot_at)
print("Angulos:", angs)

[[0.75 0.   0.25]
 [0.   0.   0.  ]
 [0.25 0.   0.75]]
Angulos: [-0.045  0.    -0.005]


In [ ]:
pot_at = np.dot(b, angs)
print("Potencia:",pot_at)

Potencia: [-0.065  0.05   0.015]


Percebe-se que, quando compara o a), com referência na barra 1, e o b), com referência na barra 2, os ângulo obtidos são diferentes mas a potência final é a mesma

##4)

### a)

In [ ]:
tipo = ['VO', 'PQ', 'PV']

# Bus columns    = 'Bus', 'Type', 'Pg', 'Pc', 'Qg', 'Qc', 'V', 'Theta', 'Bsh'
# Branch columns = 'From', 'To', 'r', 'x', 'Bsh', 'Tap'

bus = [[1, tipo[0], 0.0,   0.0, 0.0, 0.0, 1, 0, 0],
       [2, tipo[1], 0.0,  0.05, 0.0, 0.0, 1, 0, 0],
       [3, tipo[1], 0.0, 0.015, 0.0, 0.0, 1, 0, 0]]

branch = [[1, 2, 0.5, 1.0, 0.10, 1],
          [1, 3, 1.0, 2.0, 0.20, 1],
          [2, 3, 0.5, 1.0, 0.10, 1]]

i = [i+1 for i in range(len(bus))]

In [ ]:
bus_df = pd.DataFrame(bus, columns = ['Bus', 'Type', 'Pg', 'Pc', 'Qg', 'Qc', 'V', 'Theta', 'Bsh'], index = i)
bus_df

,Bus,Type,Pg,Pc,Qg,Qc,V,Theta,Bsh
1,1,VO,0.0,0.000,0.0,0.0,1,0,0
2,2,PQ,0.0,0.050,0.0,0.0,1,0,0
3,3,PQ,0.0,0.015,0.0,0.0,1,0,0


In [ ]:
branch_df = pd.DataFrame(branch, columns = ['From', 'To', 'r', 'x', 'Bsh', 'Tap'])
branch_df

,From,To,r,x,Bsh,Tap
0,1,2,0.5,1.0,0.1,1
1,1,3,1.0,2.0,0.2,1
2,2,3,0.5,1.0,0.1,1


In [ ]:
nl = newtonLinear(bus_df, branch_df, 1, False)
nl.createMatrix()
nl.angleCal()
nl.powerCal()

In [ ]:
nl.bus_df

,Bus,Type,Pg,Pc,Qg,Qc,V,Theta,Bsh
1,1,VO,0.065,0.000,0.0,0.0,1,0.000,0
2,2,PQ,0.000,0.050,0.0,0.0,1,-0.045,0
3,3,PQ,0.000,0.015,0.0,0.0,1,-0.040,0


In [ ]:
nl.branch_df

,From,To,r,x,Bsh,Tap
0,1,2,0.5,1.0,0.1,1
1,1,3,1.0,2.0,0.2,1
2,2,3,0.5,1.0,0.1,1


### b)

In [ ]:
tipo = ['VO', 'PQ', 'PV']

# Bus columns    = 'Bus', 'Type', 'Pg', 'Pc', 'Qg', 'Qc', 'V', 'Theta', 'Bsh'
# Branch columns = 'From', 'To', 'r', 'x', 'Bsh', 'Tap'

bus = [[1, tipo[0], 0.0,   0.0, 0.0, 0.0, 1, 0, 0],
       [2, tipo[1], 0.0,  0.05, 0.0, 0.0, 1, 0, 0],
       [3, tipo[1], 0.0, 0.015, 0.0, 0.0, 1, 0, 0]]

branch = [[1, 2, 0.5, 1.0, 0.10, 1],
          [1, 3, 1.0, 2.0, 0.20, 1],
          [2, 3, 0.5, 1.0, 0.10, 1]]

i = [i+1 for i in range(len(bus))]

In [ ]:
bus_df = pd.DataFrame(bus, columns = ['Bus', 'Type', 'Pg', 'Pc', 'Qg', 'Qc', 'V', 'Theta', 'Bsh'], index = i)
bus_df

,Bus,Type,Pg,Pc,Qg,Qc,V,Theta,Bsh
1,1,VO,0.0,0.000,0.0,0.0,1,0,0
2,2,PQ,0.0,0.050,0.0,0.0,1,0,0
3,3,PQ,0.0,0.015,0.0,0.0,1,0,0


In [ ]:
branch_df = pd.DataFrame(branch, columns = ['From', 'To', 'r', 'x', 'Bsh', 'Tap'])
branch_df

In [ ]:
nl = newtonLinear(bus_df, branch_df, 1, True)
nl.createMatrix()
nl.angleCal()
nl.powerCal()

In [ ]:
nl.bus_df

,Bus,Type,Pg,Pc,Qg,Qc,V,Theta,Bsh
1,1,VO,0.06614,0.000,0.0,0.0,1,0.00000,0
2,2,PQ,0.00000,0.050,0.0,0.0,1,-0.04539,0
3,3,PQ,0.00000,0.015,0.0,0.0,1,-0.04037,0


In [ ]:
nl.branch_df

,From,To,r,x,Bsh,Tap,Perdas
0,1,2,0.5,1.0,0.1,1,0.01614
1,1,3,1.0,2.0,0.2,1,0.05114
2,2,3,0.5,1.0,0.1,1,-0.06500


## 5)

### a)

In [ ]:
tipo = ['VO', 'PQ', 'PV']

# Bus columns    = 'Bus', 'Type', 'Pg', 'Pc', 'Qg', 'Qc', 'V', 'Theta', 'Bsh'
# Branch columns = 'From', 'To', 'r', 'x', 'Bsh', 'Tap'

bus = [[1, tipo[0], 0.0, 1.5, 0.0, 0.0, 1, 0, 0],
       [2, tipo[1], 0.0, 0.5, 0.0, 0.0, 1, 0, 0],
       [3, tipo[1], 0.0, 1.0, 0.0, 0.0, 1, 0, 0]]

branch = [[1, 2, 0.0, 1/3, 0.0, 1],
          [1, 3, 0.0, 1/2, 0.0, 1],
          [2, 3, 0.0, 1/3, 0.0, 1]]

i = [i+1 for i in range(len(bus))]

In [ ]:
bus_df = pd.DataFrame(bus, columns = ['Bus', 'Type', 'Pg', 'Pc', 'Qg', 'Qc', 'V', 'Theta', 'Bsh'], index = i)
bus_df

,Bus,Type,Pg,Pc,Qg,Qc,V,Theta,Bsh
1,1,VO,0.0,1.5,0.0,0.0,1,0,0
2,2,PQ,0.0,0.5,0.0,0.0,1,0,0
3,3,PQ,0.0,1.0,0.0,0.0,1,0,0


In [ ]:
branch_df = pd.DataFrame(branch, columns = ['From', 'To', 'r', 'x', 'Bsh', 'Tap'])
branch_df

,From,To,r,x,Bsh,Tap
0,1,2,0.0,0.333333,0.0,1
1,1,3,0.0,0.500000,0.0,1
2,2,3,0.0,0.333333,0.0,1


In [ ]:
nl = newtonLinear(bus_df, branch_df, 1, False)
nl.createMatrix()
nl.angleCal()
nl.powerCal()

In [ ]:
nl.bus_df

,Bus,Type,Pg,Pc,Qg,Qc,V,Theta,Bsh
1,1,VO,1.5,0.0,0.0,0.0,1,0.000000,0
2,2,PQ,0.0,0.5,0.0,0.0,1,-0.261905,0
3,3,PQ,0.0,1.0,0.0,0.0,1,-0.357143,0


In [ ]:
nl.branch_df

,From,To,r,x,Bsh,Tap
0,1,2,0.0,0.333333,0.0,1
1,1,3,0.0,0.500000,0.0,1
2,2,3,0.0,0.333333,0.0,1


### b)

$\frac{1}{x_{13}}$ passa a ser $\frac{1}{\frac{1}{2} + x}$
$$
\theta = \begin{bmatrix} \theta_1 \\ \theta_2 \\ \theta_3 \end{bmatrix}
$$
$$
P = \begin{bmatrix} P_1 \\ P_2 \\ P_3 \end{bmatrix} = \begin{bmatrix} 1.5 \\ -0.5 \\ -1.0 \end{bmatrix}
$$
$$
B^{'} = \begin{bmatrix} 3 + \frac{1}{\frac{1}{2}+x} & -3 & - \frac{1}{\frac{1}{2}+x} \\ -3 & 5 & -2 \\ - \frac{1}{\frac{1}{2}+x} & -2 & 2 + \frac{1}{\frac{1}{2}+x} \end{bmatrix}
$$

Considerando a barra 1 como referência:

$$
B^{'} = \begin{bmatrix} 5 & -2 \\ -2 & 2 + \frac{1}{\frac{1}{2}+x} \end{bmatrix}
$$
$$
\theta = \begin{bmatrix} \theta_2 \\ \theta_3 \end{bmatrix} = (B^{'})^{-1}.P = \begin{bmatrix} \frac{x+1}{3x+4} & \frac{x+0.5}{3x+4} \\ \frac{x+0.5}{3x+4} & \frac{2.5x + 1.25}{3x+4} \end{bmatrix}.\begin{bmatrix} P_2 \\ P_3 \end{bmatrix}
$$

$$
\theta = \begin{bmatrix} \frac{x+1}{3x+4} & \frac{x+0.5}{3x+4} \\ \frac{x+0.5}{3x+4} & \frac{2.5x + 1.25}{3x+4} \end{bmatrix}.\begin{bmatrix} -0.5 \\ -1 \end{bmatrix} = \begin{bmatrix} -0.5(\frac{x+1}{3x+4}) & -(\frac{x+0.5}{3x+4}) \\ -0.5(\frac{x+0.5}{3x+4}) & -(\frac{2.5x + 1.25}{3x+4}) \end{bmatrix}
$$

Portanto, lembrando que $P_{12}$ deve ser igual a 0.5:

$$
P_{12} = 0.5 = \frac{1}{x_{12}}(\theta_1 - \theta_2) = 3(0 + 0.5(\frac{x+1}{3x+4})+(\frac{x+0.5}{3x+4}))
$$
$$
0.5 = \frac{3}{3x+4}(1.5x+1)
$$
$$
1.5x + 2 = 4.5x + 3
$$
$$
x = -\frac{1}{3}
$$
Logo, a reatância do menor banco de capacitores inserido, de modo que atenda as epecificações deve ser de $-\frac{1}{3}$ pu.